In [1]:
import xml.dom.minidom
import uuid
import subprocess
import os
from tqdm import tqdm
import random 
import shutil 

In [7]:
# consts
SCENE_FILE_MASTER = 'master_v2.xml'
DATA_SET_ROOT = '/home/ax/master/2019_02/masterThesis/DeepExplain/experiments/data/generated/test'
if not os.path.exists(DATA_SET_ROOT):
    os.mkdir(DATA_SET_ROOT)
master_doc = xml.dom.minidom.parse(SCENE_FILE_MASTER)
SAMPLES = 200
X_MIN = -100
X_MAX = 100
Z_MIN = -500
Z_MAX = 0
ANGLE_MIN = -60
ANGLE_MAX = 60

POSE_ID_MIN = 1
POSE_ID_MAX = 11

master_requirements = [f'pose_0_{i}.serialized' for i in range(POSE_ID_MIN, POSE_ID_MAX)]
master_requirements.append('MikeAlger_Texture.jpeg')

for file in master_requirements:
    shutil.copy(file, f'{DATA_SET_ROOT}/{file}')
# requirements for mitsuba
my_env = os.environ.copy()
my_env["LD_LIBRARY_PATH"] = "/home/ax/Programs/mitsuba/dist:" + my_env["LD_LIBRARY_PATH"]
my_env["PATH"] = "/home/ax/Programs/mitsuba/dist:" + my_env["PATH"]

In [8]:
# compute a scene string
def get_scene_xml_string(doc, x, z, angle, pose_id):
    
    shape_node_human = doc.getElementsByTagName('shape')[0]
    transform_node = shape_node_human.getElementsByTagName('transform')[0]
    translation_node = transform_node.getElementsByTagName('translate')[0]
    rotation_node = transform_node.getElementsByTagName('rotate')[0]
    filename_node = shape_node_human.getElementsByTagName('string')[0]
    
    translation_node.setAttribute('z', str(z)) 
    translation_node.setAttribute('x', str(x))
    rotation_node.setAttribute('angle', str(angle))
    filename_node.setAttribute('value', f"pose_0_{pose_id}.serialized")
    
    shape_node_rect = doc.getElementsByTagName('shape')[1]
    transform_node = shape_node_rect.getElementsByTagName('transform')[0]
    translation_node = transform_node.getElementsByTagName('translate')[1]
    rotation_node = transform_node.getElementsByTagName('rotate')[0]
    
    translation_node.setAttribute('z', str(z)) 
    translation_node.setAttribute('x', str(x))
    rotation_node.setAttribute('angle', str(angle))
    
    return doc.toxml()

# save the scene to file
def save_scene(scene_str, pose_id, path=DATA_SET_ROOT):
    file_name = f'{path}/a_{pose_id}_{uuid.uuid4()}.xml'
    with open(file_name, "w") as f:
        f.write(scene_str)
    return file_name

# 
def render_scene(filename):
    cmd = ["/home/ax/Programs/mitsuba/build/release/mitsuba/mitsuba", filename]
    cmd_out = subprocess.run(cmd, check=True, env=my_env)
    return filename.replace('xml', 'exr')
    
def convert_exr_to_png(filename):
    cmd = ["convert", filename, '-colorspace', 'RGB', '-colorspace', 'sRGB', filename.replace('exr', 'jpg')]
    cmd_out = subprocess.run(cmd, check=True, env=my_env)
    
def clean_up(filename):
    os.remove(filename)
    
def generate_sample(master_doc, x, z, angle, pose_id):
    # generate xml scene string
    scene_str = get_scene_xml_string(master_doc, x, z, angle, pose_id)
    # save scene as file
    scene_file_name = save_scene(scene_str, pose_id)
    # render scene to exr
    result_file_name = render_scene(scene_file_name)
    # convert to png
    convert_exr_to_png(result_file_name)
    # delete exr
    clean_up(result_file_name)

In [9]:
for i in tqdm(range(SAMPLES)):
    x = random.uniform(X_MIN, X_MAX)
    z = random.uniform(Z_MIN, Z_MAX)
    angle = random.uniform(ANGLE_MIN, ANGLE_MAX)
    pose_id = random.randint(POSE_ID_MIN, POSE_ID_MAX - 1)
    
    generate_sample(master_doc, x, z, angle, pose_id)

100%|██████████| 200/200 [09:44<00:00,  2.92s/it]


In [22]:
generate_sample(master_doc, x, z, angle, 1)